In [1]:
# v1: lr = 5e-1;  factor=0.3, patience=5,
# v2: lr = 1e-1;  factor=0.3, patience=5,
# v3: lr = 8e-1;  factor=0.3, patience=5, 没改上
# v4: lr = 8e-1;  factor=0.3, patience=5,
# v5: lr = 1.5;  factor=0.2, patience=5,
# v6: lr = 8e-1;  cosnealling warm
# v7: 'seresnet152d' lr=8e-1 factor=0.3, patience=5
# v8: 'seresnet152d' lr=1e-1 factor=0.3, patience=5
# v11: 'seresnet152d' lr=1 factor=0.3, patience=5
# v12: 'seresnet152d' lr=1.5 factor=0.3, patience=5
# v13: normalize( *std + mean) label lr=1e-4
# v14: normalize(/mean) label lr=1e-4

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import torch
import time
import copy
import sys
sys.path.append(r'../input/pytorch-image-models/pytorch-image-models-master/')
import timm
import torch
import torch.nn as nn
import random
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error
from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler

## CFG

In [3]:
# ====================================================
# CFG
# ====================================================
class CFG:
    
    # 路径
    OUTPUT_DIR = './'
    DATA_ROOT = '../input/1st-reproduce-mel-spectrogram/train'
    
    model_name='seresnet152d'#'resnext50_32x4d'
    target_size = 1
    seed = 22
    split_ratio = 0.33
    num_workers = 1
    epochs = 300
    EARLY_STOP = True
    early_stop = 20
    lr = 1e-4
    min_lr = 1e-5
    weight_decay = 1e-6
    batch_size = 32
    
    scheduler='ReduceLROnPlateau'#'CosineAnnealingWarmRestarts'#
    target_col = 'time_to_eruption_normalize'
    # for warm start
    T_0 = 10
    # for normal cosine
    T_max = 10
    
    gradient_accumulation_steps=1
    max_grad_norm = 1000
    print_freq = 20
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
seed_everything(CFG.seed)

In [6]:
# model_name设置
# resnet: 'resnet50'
# resnext: 'resnext50_32x4d'
# resnest: 'resnest50d'
# seresnet: 'seresnet50'

class MyResNet(nn.Module):
    def __init__(self, target_size, model_name, pretrained=False):
        super(MyResNet, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        n_features = self.model.fc.in_features
        # 改成自己任务的图像类别数
        self.model.fc = nn.Linear(n_features, target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
# train valid split
file = os.listdir(r'../input/1st-reproduce-mel-spectrogram/train')

X_train, X_test, y_train, y_test = train_test_split(np.arange(len(file)).reshape(-1,1), 
                                                    np.arange(len(file)).reshape(-1,1), test_size=CFG.split_ratio, random_state=CFG.seed)

# train和valid的file名字
# train_file = [file[i] for i in X_train.ravel().astype(int)]
# valid_file = [file[i] for i in X_test.ravel().astype(int)]

# # id2filename的字典，用于快速索引
# trainidx2file = dict(zip(list(range(len(train_file))), train_file))
# valididx2file = dict(zip(list(range(len(valid_file))), valid_file))

df = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/train.csv')
time2eruption_mean = df['time_to_eruption'].mean()
# time2eruption_std = df['time_to_eruption'].std()
# df['time_to_eruption_normalize'] = (df['time_to_eruption'] - time2eruption_mean) / time2eruption_std
df['time_to_eruption_normalize'] = df['time_to_eruption'] / time2eruption_mean
# # segmentid
# train_name = [int(i[:-9]) for i in train_file]
# valid_name = [int(i[:-9]) for i in valid_file]

# segment_id和time组成的两列，target df
train_df = df.loc[X_train.ravel()].reset_index(drop=True)
valid_df = df.loc[X_test.ravel()].reset_index(drop=True)
# # train_df = df.loc[df['segment_id'].isin(train_name)].reset_index(drop=True)
# # valid_df = df.loc[df['segment_id'].isin(valid_name)].reset_index(drop=True)

# tmp1 = pd.DataFrame({'idx': trainidx2file.keys(), 'file': trainidx2file.values()})
# tmp1['segment_id'] = tmp1['file'].map(lambda x: int(x[:-9]))

# tmp2 = pd.DataFrame({'idx': valididx2file.keys(), 'file': valididx2file.values()})
# tmp2['segment_id'] = tmp2['file'].map(lambda x: int(x[:-9]))

# train_df = pd.merge(train_df, tmp1, on='segment_id', how='left')
# valid_df = pd.merge(valid_df, tmp2, on='segment_id', how='left')

# train_df.sort_values(by='idx', ascending=True, inplace=True)
# valid_df.sort_values(by='idx', ascending=True, inplace=True)

# # idx 对应的 df
# train_df = train_df[['segment_id', 'time_to_eruption']].reset_index(drop=True)
# valid_df = valid_df[['segment_id', 'time_to_eruption']].reset_index(drop=True)

In [8]:
class INGVDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df
        self.data_root = data_root
        self.transforms = transforms
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            #target = self.df.iloc[index]['label']
            target = [self.df.loc[index, 'time_to_eruption_normalize'].astype('float32')]
          
        path = os.path.join(self.data_root, str(self.df.loc[index, 'segment_id'])+'-spec.npz')
        
        # (10, 256, 256)
        img = np.load(path)['arr_0'].astype('float32')
            
        # do label smoothing
        if self.output_label == True:
            return torch.tensor(img), torch.tensor(target)
        else:
            return torch.tensor(img)

In [9]:
# 记录训练过程
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        
        self.true_val = 0
        self.true_avg = 0
        self.true_sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
    def true_update(self, val, n=1):
        self.true_val = val * time2eruption_mean
        self.true_sum += self.true_val * n
        self.count += n
        self.true_avg = self.true_sum / self.count

In [10]:
def asMinutes(s):
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [11]:
def train_fn(train_loader, model, epoch, criterion, optimizer, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    # switch to train mode
    model.train()
    
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        # torch.Size([32, 10, 256, 256])
        # print(images.shape)
        # torch.Size([32, 1])
        # print(labels.shape)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(images)
        # torch.Size([32, 1])
        # print(y_preds.shape)
        loss = criterion(y_preds, labels)
        # record loss
        losses.update(loss.item(), batch_size)
        losses.true_update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) {loss.true_val:.4f}({loss.true_avg:.4f})'
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        losses.true_update(loss.item(), batch_size)
        # record accuracy
        # preds.append(y_preds.softmax(1).to('cpu').numpy())
        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) {loss.true_val:.4f}({loss.true_avg:.4f})'
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    
    return losses.avg, predictions

In [12]:
def init_logger(log_file=CFG.OUTPUT_DIR+'train.log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

In [13]:
def get_score(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

In [14]:
# ====================================================
# Train loop
# ====================================================

LOGGER = init_logger()

def train_loop(train_df, valid_df):

    LOGGER.info(f"========== start training ==========")
    
    # 存储结果
    res_valid_df = copy.deepcopy(valid_df)
    # ====================================================
    # loader
    # ====================================================

    train_dataset = INGVDataset(train_df, CFG.DATA_ROOT)
    valid_dataset = INGVDataset(valid_df, CFG.DATA_ROOT)

    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=False)
    
    # model
    model = MyResNet(CFG.target_size, CFG.model_name, True)
    model.to(CFG.device)
    
    # optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    
    # ====================================================
    # scheduler 
    # ====================================================
    if CFG.scheduler=='CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=2, eta_min=CFG.min_lr, last_epoch=-1)
    elif CFG.scheduler=='ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
    else:
        raise Exception('scheduler not set')


    # ====================================================
    # loop
    # ====================================================
    criterion = nn.L1Loss()

    best_score = float('inf')
    best_loss = np.inf
    early_round = 0
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # =================================================
        # train
        avg_loss = train_fn(train_loader, model, epoch, criterion, optimizer, scheduler, CFG.device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, CFG.device)
        valid_labels = valid_df[CFG.target_col].values
        
        if isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingWarmRestarts):
            scheduler.step()
        elif isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - MAE: {score}')
        
#         save_checkpoint(model, optimizer, scheduler, epoch, fold)
#         print('===================== checkpoint saved =======================')
        
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        CFG.OUTPUT_DIR+f'{CFG.model_name}_best.pth')
            early_round = 0
            
        else:
            if CFG.EARLY_STOP:
                early_round += 1
                if early_round == CFG.early_stop:
                    LOGGER.info('===================== Early Stop =====================')
                    break
    
    check_point = torch.load(CFG.OUTPUT_DIR+f'{CFG.model_name}_best.pth')
    res_valid_df['preds'] = check_point['preds']

    return res_valid_df

In [15]:
res_valid_df = train_loop(train_df, valid_df)

========== start training ==========


Epoch: [1][0/93] Data 1.110 (1.110) Elapsed 0m 4s (remain 6m 37s) Loss: 0.6758(0.6758) 15442050.3982(7721025.1991)Grad: 11.3421  
Epoch: [1][20/93] Data 0.015 (0.068) Elapsed 0m 21s (remain 1m 13s) Loss: 0.4255(0.2627) 9722374.0917(5859445.0745)Grad: 4.2857  
Epoch: [1][40/93] Data 0.021 (0.043) Elapsed 0m 38s (remain 0m 49s) Loss: 0.3249(0.2379) 7423321.1364(5368940.2184)Grad: 6.3193  
Epoch: [1][60/93] Data 0.015 (0.034) Elapsed 0m 56s (remain 0m 29s) Loss: 0.4119(0.2245) 9410412.9686(5088533.6242)Grad: 7.3536  
Epoch: [1][80/93] Data 0.014 (0.030) Elapsed 1m 13s (remain 0m 10s) Loss: 0.3149(0.2173) 7195994.6684(4935552.7995)Grad: 7.5141  
Epoch: [1][92/93] Data 0.015 (0.028) Elapsed 1m 25s (remain 0m 0s) Loss: 0.3755(0.2143) 8579471.5599(4877177.3611)Grad: 10.5113  
EVAL: [0/46] Data 0.970 (0.970) Elapsed 0m 1s (remain 0m 52s) Loss: 0.3228(0.3228) 7374667.9050(3687333.9525)
EVAL: [20/46] Data 0.529 (0.578) Elapsed 0m 16s (remain 0m 19s) Loss: 0.3054(0.1893) 6978045.5697(4223008.8094

Epoch 1 - avg_train_loss: 0.2143  avg_val_loss: 0.1859  time: 118s
Epoch 1 - MAE: 0.3687935446147274
Epoch 1 - Save Best Score: 0.3688 Model


EVAL: [45/46] Data 0.320 (0.515) Elapsed 0m 33s (remain 0m 0s) Loss: 0.2843(0.1859) 6495031.7269(4213264.6925)
Epoch: [2][0/93] Data 0.368 (0.368) Elapsed 0m 1s (remain 1m 51s) Loss: 0.3406(0.3406) 7783282.4593(3891641.2297)Grad: 5.7537  
Epoch: [2][20/93] Data 0.014 (0.032) Elapsed 0m 18s (remain 1m 1s) Loss: 0.3744(0.1855) 8554834.0903(4137338.5744)Grad: 5.2144  
Epoch: [2][40/93] Data 0.015 (0.024) Elapsed 0m 34s (remain 0m 43s) Loss: 0.3638(0.1833) 8312723.5063(4136695.9197)Grad: 4.6810  
Epoch: [2][60/93] Data 0.014 (0.021) Elapsed 0m 51s (remain 0m 26s) Loss: 0.3573(0.1797) 8164355.2903(4073242.6845)Grad: 8.3575  
Epoch: [2][80/93] Data 0.014 (0.019) Elapsed 1m 7s (remain 0m 10s) Loss: 0.3200(0.1764) 7310723.2494(4005819.5712)Grad: 6.4013  
Epoch: [2][92/93] Data 0.016 (0.019) Elapsed 1m 17s (remain 0m 0s) Loss: 0.4056(0.1745) 9266447.0975(3971584.1855)Grad: 7.9338  
EVAL: [0/46] Data 0.364 (0.364) Elapsed 0m 0s (remain 0m 25s) Loss: 0.2702(0.2702) 6173241.7645(3086620.8822)
EVAL

Epoch 2 - avg_train_loss: 0.1745  avg_val_loss: 0.1697  time: 92s
Epoch 2 - MAE: 0.336798385022828
Epoch 2 - Save Best Score: 0.3368 Model


...

EVAL: [45/46] Data 0.042 (0.159) Elapsed 0m 16s (remain 0m 0s) Loss: 0.0261(0.0147) 597319.4684(333213.4301)
Epoch: [254][0/93] Data 0.387 (0.387) Elapsed 0m 1s (remain 1m 51s) Loss: 0.0161(0.0161) 367854.7308(183927.3654)Grad: 8.8418  
Epoch: [254][20/93] Data 0.014 (0.033) Elapsed 0m 17s (remain 1m 0s) Loss: 0.0108(0.0054) 246311.0483(121496.6632)Grad: 2.0438  
Epoch: [254][40/93] Data 0.015 (0.025) Elapsed 0m 34s (remain 0m 43s) Loss: 0.0102(0.0058) 232513.7360(130984.4279)Grad: 1.6565  
Epoch: [254][60/93] Data 0.015 (0.022) Elapsed 0m 51s (remain 0m 26s) Loss: 0.0137(0.0059) 312849.3379(133505.2650)Grad: 12.2568  
Epoch: [254][80/93] Data 0.017 (0.021) Elapsed 1m 8s (remain 0m 10s) Loss: 0.0112(0.0060) 256566.8224(135184.0564)Grad: 7.8915  
Epoch: [254][92/93] Data 0.014 (0.020) Elapsed 1m 18s (remain 0m 0s) Loss: 0.0105(0.0059) 239171.0484(135318.2863)Grad: 4.0142  
EVAL: [0/46] Data 0.376 (0.376) Elapsed 0m 0s (remain 0m 25s) Loss: 0.0227(0.0227) 518511.8342(259255.9171)
EVAL: [

Epoch 254 - avg_train_loss: 0.0059  avg_val_loss: 0.0147  time: 94s
Epoch 254 - MAE: 0.029086192886820492


EVAL: [45/46] Data 0.042 (0.141) Elapsed 0m 15s (remain 0m 0s) Loss: 0.0250(0.0147) 570625.2730(332293.8519)
Epoch: [255][0/93] Data 0.382 (0.382) Elapsed 0m 1s (remain 2m 3s) Loss: 0.0103(0.0103) 234864.4622(117432.2311)Grad: 3.7523  
Epoch: [255][20/93] Data 0.016 (0.033) Elapsed 0m 17s (remain 1m 1s) Loss: 0.0135(0.0053) 308744.4194(118859.8380)Grad: 4.6672  
Epoch: [255][40/93] Data 0.022 (0.025) Elapsed 0m 34s (remain 0m 44s) Loss: 0.0118(0.0054) 268745.4303(121398.9572)Grad: 6.8136  
Epoch: [255][60/93] Data 0.015 (0.022) Elapsed 0m 51s (remain 0m 26s) Loss: 0.0096(0.0054) 220342.0013(122224.9786)Grad: 1.5045  
Epoch: [255][80/93] Data 0.016 (0.021) Elapsed 1m 8s (remain 0m 10s) Loss: 0.0074(0.0055) 168128.8459(125748.3312)Grad: 3.5793  
Epoch: [255][92/93] Data 0.015 (0.021) Elapsed 1m 18s (remain 0m 0s) Loss: 0.0178(0.0055) 406693.9335(125314.2662)Grad: 7.5514  
EVAL: [0/46] Data 0.401 (0.401) Elapsed 0m 0s (remain 0m 27s) Loss: 0.0234(0.0234) 534533.8331(267266.9166)
EVAL: [20

Epoch 255 - avg_train_loss: 0.0055  avg_val_loss: 0.0147  time: 95s
Epoch 255 - MAE: 0.029249663605657894
===================== Early Stop =====================


EVAL: [45/46] Data 0.040 (0.154) Elapsed 0m 16s (remain 0m 0s) Loss: 0.0261(0.0147) 597083.8595(334161.4113)


In [16]:
res_valid_df.head()

,segment_id,time_to_eruption,time_to_eruption_normalize,preds
0,859980212,17523064,0.766910,0.781434
1,925884391,41682685,1.824275,1.828830
2,2143227248,28548623,1.249452,1.245996
3,2029121195,43190417,1.890262,1.892455
4,586194492,38850589,1.700326,1.735031


In [17]:
res_valid_df.to_pickle('res_valid_df.pkl')